In [2]:
%load_ext bigdata
%hive_start
%timeout 300

In [3]:
%%hive
DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
--
DROP TABLE IF EXISTS tbl1;
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 10.804 seconds
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 1.227 seconds
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 2.443 seconds
--
DROP TABLE IF EXISTS tbl1;
OK
Time taken: 0.01 seconds
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 0.084 seconds
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;
Loading data to table default.tbl1
OK
Time taken: 1.453 seconds


In [21]:
%%hive 
DROP TABLE IF EXISTS dataT; 
CREATE TABLE dataT (num INT, clave STRING, valor INT);
INSERT OVERWRITE TABLE dataT
SELECT
    c1,
    clave,
    valor
FROM
    tbl1
LATERAL VIEW
    explode(c4) tbl1 AS clave,valor;

DROP TABLE IF EXISTS dataT; 
OK
Time taken: 0.008 seconds
CREATE TABLE dataT (num INT, clave STRING, valor INT);
OK
Time taken: 0.066 seconds
INSERT OVERWRITE TABLE dataT
SELECT
    c1,
    clave,
    valor
FROM
    tbl1
LATERAL VIEW
    explode(c4) tbl1 AS clave,valor;
Query ID = root_20200228155510_1a4660a0-70d6-450d-80a6-05452feeb5ad
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1582902829190_0013, Tracking URL = http://5c70801abc21:8088/proxy/application_1582902829190_0013/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582902829190_0013
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-02-28 15:55:18,913 Stage-1 map = 0%,  reduce = 0%
2020-02-28 15:55:24,489 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 1.67 sec
MapReduce Total cumulative CPU time: 1 seconds 670 msec
Ended Job = job_1582902829190_0013
Stage-4 is selected by condition resolver.
Stage-3

In [20]:
%%hive
SELECT c1, c2 FROM tbl0 LIMIT 5;

SELECT c1, c2 FROM tbl0 LIMIT 5;
OK
1	D
2	C
3	D
4	D
5	C
Time taken: 0.207 seconds, Fetched: 5 row(s)


In [26]:
%%hive

SELECT
    d.c1,
    d.c2,
    t.valor
FROM
    tbl0 d
JOIN (
    SELECT
        num,
        clave,
        valor
    FROM
        dataT
    ) t
ON
    (d.c2 = t.clave AND d.c1 = t.num);

SELECT
    d.c1,
    d.c2,
    t.valor
FROM
    tbl0 d
JOIN (
    SELECT
        num,
        clave,
        valor
    FROM
        dataT
    ) t
ON
    (d.c2 = t.clave AND d.c1 = t.num);
Query ID = root_20200228160242_71bff50e-d0bf-49a5-b328-d76fb40aa1c7
Total jobs = 1
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
2020-02-28 16:02:52	Starting to launch local task to process map join;	maximum memory = 477626368
2020-02-28 16:02:54	Dump the side-table for tag: 1 with group count: 50 into file: file:/tmp/root/d5616ae2-31fb-4b6c-a042-82057907b08d/hive_2020-02-28_16

In [27]:
%hive_quit

In [28]:
## Copia Archivos a sistema HDFS
##
## Se usan un directorio temporal en el HDFS. La siguiente
## instrucción muestra el contenido del dicho directorio
##
!hdfs dfs -ls /tmp

Found 4 items
drwxrwx---   - root supergroup          0 2020-02-28 15:14 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-28 15:19 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-28 15:40 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-02-28 15:33 /tmp/wordcount


In [29]:
## Crea la carpeta wordcount en el hdfs
##
!hdfs dfs -mkdir /tmp/wordcount

mkdir: `/tmp/wordcount': File exists


In [30]:
## Verifica la creación de la carpeta
##
!hdfs dfs -ls /tmp/

Found 4 items
drwxrwx---   - root supergroup          0 2020-02-28 15:14 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-28 15:19 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-28 15:40 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-02-28 15:33 /tmp/wordcount


In [31]:
## Copia los archvios del directorio local wordcount/
## al directorio /tmp/wordcount/ en el hdfs
##
!hdfs dfs -copyFromLocal 'data.tsv'  /tmp/wordcount/

copyFromLocal: `data.tsv': No such file or directory


In [32]:
##
## Verifica que los archivos esten copiados
## en el hdfs
##
!hdfs dfs -ls /tmp/wordcount

Found 1 items
-rw-r--r--   1 root supergroup       1394 2020-02-28 15:33 /tmp/wordcount/data.tsv


In [33]:
####################################################################################################################
## Generación del script y ajuste del código

In [34]:
%%writefile wordcount.hql

DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
--
DROP TABLE IF EXISTS tbl1;
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;




DROP TABLE IF EXISTS dataT; 
CREATE TABLE dataT (num INT, clave STRING, valor INT);
INSERT OVERWRITE TABLE dataT
SELECT
    c1,
    clave,
    valor
FROM
    tbl1
LATERAL VIEW
    explode(c4) tbl1 AS clave,valor;
    
INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','

SELECT
    d.c1,
    d.c2,
    t.valor
FROM
    tbl0 d
JOIN (
    SELECT
        num,
        clave,
        valor
    FROM
        dataT
    ) t
ON
    (d.c2 = t.clave AND d.c1 = t.num);

Writing wordcount.hql


In [35]:
## Ejecucion
!hive -S -e 'source wordcount.hql'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


In [36]:
### Copia de los resultados a la maquina local (vagrant)
!hadoop fs -copyToLocal /tmp/output output
!ls output/*

output/000000_0


In [37]:
!cat output/000000_0 | head

1,D,3
2,C,0
3,D,1
4,D,5
5,C,2
6,A,4
7,B,5
8,C,5
9,B,4
10,B,3
